# Assignment 5 

## Grade: /100 pts

This notebook contains the questions for Assignment 5. 

Please note, a random seed has been set (42) to ensure the reproducibility of the results -- *DO NOT* change this random seed. **If you call additional functions that are based on random number generators, you will need to define their seed to 42 as well**. 

Make sure to complete this assignment individually and appropriately reference all external code and documentation used. ***In order for your submission to be valid, you must adhere to the function definitions which have been made (failure to do so will result in a grade of 0). You must upload this completed Jupyter Notebook file as your submission (other file types are not permitted and will result in a grade of 0).*** You are responsible for selecting and importing additional packages.

In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import time
%matplotlib inline

## Q1: Data loading and Preprocessing (20 pts)

In this assignment we use a dataset (`data.csv`) to diagnose breast cancer as malignant (`M`) or benign (`B`):

- Load the dataset and show the first 5 rows
- Remove `id` from the dataframe
- Remove rows with any null values
- Seperate the target variable `diagnosis` and feature matrix.
* Use label encoder to make the labels numerical (0 and 1 instead of `B` and `M`).
* Split the data and set aside 10 percent of data for testing.
* Print the size of training and test sets.

## Q2: Effect of normalization (15 pts)

In order to properly build ML models, the features need to be standardized so that no feature can dominate others in determining the prediction values due to differences in feature scales. 

* Standarize training and test sets and plot the `area_mean` feature before and after the standardization. 

* **Written Answer:** Make observations about the plot.  

**Written Answer:**

## Q3: Classification model1 (15 pts)

* Fit a logistic regression model (model1) to the normalized training data and report the training time.
* Report the accuracy and recall for model1's predictions. 

## Q4: PCA (15 pts)

In this task:

- Use the normalized training data and plot the explained variance ratio for the data.
- **Written Answer** Use that plot to choose the proper number of principal components (PCs) which can explain at least 70% of the total variance in the training data. Report the number of your choice and explain how you have chosen that number?

**Written Answer:**

## Q5: Dimensionality Reduction (15 pts)

* Project the scaled training data using PCA with the dimension of 2. 
* Plot the projected training dataponts on the PCs and show different labels with two different colors. 

## Q6: Classification model2 (on compressed data) (20 pts)

* Fit a new logistic regression model (model2) and fit it to the compressed training data from Q5 and report the training time.
* Project the normalized test set using PCA model you created in Q5 and report the accuracy and recall for model2's prediction. 
* **Written Answer** compare two models (model1 and model2) in terms of training time and prediction results. What do you conclude?